In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from kss import split_sentences
import faiss
import numpy as np
from sklearn.neighbors import NearestNeighbors



In [2]:
import torch

In [3]:
model_path = '/home/u4026/KoSentenceBERT_SKTBERT/output/training_sts'
model = SentenceTransformer(model_path)

using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_v1.zip
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Load Model


/home/u4026/.local/lib/python3.8/site-packages/sentence_transformers/models/Transformer.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.auto_model.load_state_dict(

In [4]:
nli_model_path = '/home/u4026/KoSentenceBERT_SKTBERT/output/training_nli'
nli_model = SentenceTransformer(nli_model_path)

using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_v1.zip
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Load Model


In [5]:
# CSV 파일 로드
csv_file = "books_summary.csv"
data = pd.read_csv(csv_file)

In [6]:
# 전처리된 텍스트 열 추출
corpus = data['내용요약'].tolist()
titles = data['첵제목'].tolist()

In [7]:
# 내용요약을 문장별로 나누기
def split_corpus(corpus):
    split_sentences_corpus = []
    mapping = []  # 원본 데이터의 인덱스 매핑
    for idx, text in enumerate(corpus):
        sentences = split_sentences(text)
        split_sentences_corpus.extend(sentences)
        mapping.extend([idx] * len(sentences))
    return split_sentences_corpus, mapping

In [8]:
# 평균 임베딩 계산
def calculate_average_embedding(sentences, model):
    embeddings = model.encode(sentences, convert_to_tensor=True)
    return embeddings.mean(dim=0)


In [17]:
#후보군 찾기 (STS 활용, 평균 임베딩)
def find_candidates_with_sts_average(query, split_corpus, mapping, titles, n_neighbors=3):
    average_embeddings = []
    unique_indices = list(set(mapping))

    for idx in unique_indices:
        sentences = [split_corpus[i] for i in range(len(mapping)) if mapping[i] == idx]
        avg_embedding = calculate_average_embedding(sentences, model)
        average_embeddings.append(avg_embedding)

    average_embeddings = torch.stack(average_embeddings)
    query_embedding = calculate_average_embedding([query], model)

    # 코사인 유사도 계산
    similarities = util.pytorch_cos_sim(query_embedding, average_embeddings)
    top_indices = torch.topk(similarities[0], k=n_neighbors, largest=True).indices.tolist()

    candidates = []
    for idx in top_indices:
         if isinstance(idx, int):
            original_idx = unique_indices[idx]
            candidates.append({
              "책제목": titles[original_idx],
              "내용요약": corpus[original_idx],
               "유사도": similarities[0][idx].item()
        })
    return sorted(candidates, key=lambda x: x["유사도"], reverse=True)

In [20]:
# 후보군 찾기 (KNN 활용)
def find_candidates_with_knn(query, split_corpus, mapping, titles, n_neighbors=3):
    # STS 임베딩 생성
    corpus_embeddings = model.encode(split_corpus)
    query_embedding = model.encode([query])

    # Nearest Neighbors 모델 초기화 및 학습
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn.fit(corpus_embeddings)

    # Query와 가장 가까운 이웃 찾기
    distances, indices = knn.kneighbors(query_embedding)

    candidates = []
    for dist, idx in zip(distances[0], indices[0]):
        original_idx = mapping[idx]
        candidates.append({
            "책제목": titles[original_idx],
            "내용요약": split_corpus[idx],
            "유사도": 1 - dist  # 코사인 거리 -> 유사도로 변환
        })

    # 유사도를 기준으로 정렬
    return sorted(candidates, key=lambda x: x["유사도"], reverse=True)


In [14]:
# 최종 유사도 평가 (STS 활용)
def evaluate_with_sts(query, candidates):
    query_embedding = model.encode([query], convert_to_tensor=True)
    results = []

    for candidate in candidates:
        corpus_embedding = model.encode([candidate['내용요약']], convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(query_embedding, corpus_embedding).item()

        results.append({
            "책제목": candidate['책제목'],
            "내용요약": candidate['내용요약'],
            "유사도": similarity
        })

    # 유사도를 기준으로 정렬
    return sorted(results, key=lambda x: x["유사도"], reverse=True)

In [19]:

# 최종 유사도 평가 (NLI 활용)
def evaluate_with_nli(query, candidates):
    query_embedding = nli_model.encode([query], convert_to_tensor=True)
    results = []

    for candidate in candidates:
        corpus_embedding = nli_model.encode([candidate['내용요약']], convert_to_tensor=True)
        #top_indices = torch.topk(similarities[0], k=n_neighbors, largest=True).indices.tolist()
        similarity = util.pytorch_cos_sim(query_embedding, corpus_embedding).item()

        results.append({
            "책제목": candidate['책제목'],
            "내용요약": candidate['내용요약'],
            "유사도": similarity
        })
    return sorted(results, key=lambda x: x["유사도"], reverse=True)

In [22]:
# 사용자가 입력한 텍스트
query = input("텍스트를 입력하세요: ")


텍스트를 입력하세요: 새 교육과정에 따른 평가기준에 관한 내용의 책 추천해줘


In [23]:
# Step 1: 내용요약을 문장별로 나누기
split_corpus_data, mapping = split_corpus(corpus)

# Step 2: STS으로 후보군 찾기
candidates = find_candidates_with_sts_average(query, split_corpus_data, mapping, titles, n_neighbors=3)

# Step 3: nli로 최종 유사도 계산
final_results = evaluate_with_nli(query, candidates)

print(final_results)


[{'책제목': '교육광장 2016 겨울호 (Vol. 62)', '내용요약': '새 교육과정에 따른 평가기준의 개발이 필요해서, 한국교육과정평가원에서는 총론과 교과별 평가기준 등을 연구 개발 하였다.  핵심 개발 방향은 2015년 개정 교육과정이 학교현장에서 운영되도록 하는 것이다.', '유사도': 0.9067184925079346}, {'책제목': '2013년 국가수준 학업성취도 평가 결과: 인지적·정의적 특성 및 변화 추이', '내용요약': '우리나라 학생들이 의사결정 영향력을 가장 체감하는 영역은 소풍과 수학여행 장소 결정 등으로 매우 제한적인 반면, 다른 나라들의 경우 학생들이 교육과정 운영, 교사 채용 과정, 학교 규정 등에 영향력을 행사한다. 교육과정은 학교의 핵심 활동인 만큼 교원능력개발평가 등을 통해 교육과정 선정 과정에서 학생 참여를 증대할 필요가 있다. 이를 위해서는 현재의 교원능력개발평가문항을 보완하여 학생들이 수업에 대한 의견을 제시할 수 있도록 해야 한다.', '유사도': 0.8830466866493225}, {'책제목': '교육정책포럼 2015년 07월 (통권 265호)', '내용요약': '개정 교육과정의 운영 방법을 명확히 하기 위해 전반적으로 학생들의 학습 강점과 요구사항 등을 교육과정에 적절히 반영하였다.', '유사도': 0.8808505535125732}]


In [27]:
for result in final_results:
    print(f"책제목: {result['책제목']}\n내용요약: {result['내용요약']}\n유사도: {result['유사도']}\n")

책제목: 교육광장 2016 겨울호 (Vol. 62)
내용요약: 새 교육과정에 따른 평가기준의 개발이 필요해서, 한국교육과정평가원에서는 총론과 교과별 평가기준 등을 연구 개발 하였다.  핵심 개발 방향은 2015년 개정 교육과정이 학교현장에서 운영되도록 하는 것이다.
유사도: 0.9067184925079346

책제목: 2013년 국가수준 학업성취도 평가 결과: 인지적·정의적 특성 및 변화 추이
내용요약: 우리나라 학생들이 의사결정 영향력을 가장 체감하는 영역은 소풍과 수학여행 장소 결정 등으로 매우 제한적인 반면, 다른 나라들의 경우 학생들이 교육과정 운영, 교사 채용 과정, 학교 규정 등에 영향력을 행사한다. 교육과정은 학교의 핵심 활동인 만큼 교원능력개발평가 등을 통해 교육과정 선정 과정에서 학생 참여를 증대할 필요가 있다. 이를 위해서는 현재의 교원능력개발평가문항을 보완하여 학생들이 수업에 대한 의견을 제시할 수 있도록 해야 한다.
유사도: 0.8830466866493225

책제목: 교육정책포럼 2015년 07월 (통권 265호)
내용요약: 개정 교육과정의 운영 방법을 명확히 하기 위해 전반적으로 학생들의 학습 강점과 요구사항 등을 교육과정에 적절히 반영하였다.
유사도: 0.8808505535125732

